In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import tensorflow.keras.backend as k
import re
import sqlalchemy as sqlal
import string
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, scoped_session
from sqlalchemy.sql import text

In [2]:

cn=['user_id','item_id','rate','timestep']
ratings=pd.read_csv('u.data',sep='\t',
                names=cn)
movies=pd.read_csv('Movie_Id_Titles')

cols=movies['title'].tolist()
new_cols=list(map(lambda x:re.sub('[^a-zA-Z0-9 ]',"",x).replace(" ","_"),cols))
movies['title']=new_cols
movies.head(10)

,item_id,title
0,1,Toy_Story_1995
1,2,GoldenEye_1995
2,3,Four_Rooms_1995
3,4,Get_Shorty_1995
4,5,Copycat_1995
5,6,Shanghai_Triad_Yao_a_yao_yao_dao_waipo_qiao_1995
6,7,Twelve_Monkeys_1995
7,8,Babe_1995
8,9,Dead_Man_Walking_1995
9,10,Richard_III_1995


In [4]:
df = pd.merge(ratings,movies,on='item_id')
table = pd.pivot_table(df, values='rate', index=['user_id'],columns=['title'])
table.head(10)

title,101_Dalmatians_1996,12_Angry_Men_1957,187_1997,1900_1994,20000_Leagues_Under_the_Sea_1954,2001_A_Space_Odyssey_1968,2_Days_in_the_Valley_1996,39_Steps_The_1935,3_Ninjas_High_Noon_At_Mega_Mountain_1998,8_12_1963,...,Yankee_Zulu_1994,Year_of_the_Horse_1997,You_So_Crazy_1994,Young_Frankenstein_1974,Young_Guns_1988,Young_Guns_II_1990,Young_Poisoners_Handbook_The_1995,Zeus_and_Roxanne_1997,_kldum_klaka_Cold_Fever_1994,unknown
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,NaN,4.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,4.0
6,NaN,4.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,4.0,NaN,NaN,5.0,5.0,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,3.0,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#sort columns
sorted_colmns=table.isna().sum().sort_values(ascending=True)
table_sort_by_col=table[sorted_colmns.index.values.tolist()]
#sort rows
sorted_ind=table_sort_by_col.isna().sum(axis=1)
table_sort_by_col['sorted_ind']=sorted_ind
table_sort_by_col_row=table_sort_by_col.sort_values(by=['sorted_ind'])
table_sort_by_col_row=table_sort_by_col_row.drop(['sorted_ind'], axis=1)

C:\ProgramData\Anaconda3\envs\daryl\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
data=table_sort_by_col_row.iloc[:300,:100]

In [6]:
data.head(10)

title,Star_Wars_1977,Contact_1997,Fargo_1996,Return_of_the_Jedi_1983,Liar_Liar_1997,English_Patient_The_1996,Scream_1996,Toy_Story_1995,Air_Force_One_1997,Independence_Day_ID4_1996,...,Apocalypse_Now_1979,Clockwork_Orange_A_1971,Courage_Under_Fire_1996,Lion_King_The_1994,Top_Gun_1986,Babe_1995,Volcano_1997,Aladdin_1992,To_Kill_a_Mockingbird_1962,Murder_at_1600_1997
user_id,,,,,,,,,,,,,,,,,,,,,
405,5.0,NaN,NaN,5.0,NaN,NaN,5.0,NaN,NaN,NaN,...,3.0,1.0,NaN,1.0,1.0,4.0,NaN,3.0,5.0,NaN
655,4.0,2.0,3.0,3.0,3.0,3.0,3.0,2.0,3.0,3.0,...,NaN,4.0,3.0,NaN,2.0,3.0,NaN,NaN,4.0,NaN
13,5.0,4.0,5.0,5.0,2.0,3.0,1.0,3.0,1.0,5.0,...,5.0,2.0,1.0,4.0,5.0,4.0,3.0,5.0,5.0,3.0
450,5.0,4.0,4.0,4.0,4.0,4.0,3.0,4.0,4.0,3.0,...,4.0,5.0,4.0,3.0,5.0,NaN,NaN,3.0,5.0,4.0
276,5.0,5.0,5.0,5.0,4.0,NaN,4.0,5.0,4.0,4.0,...,5.0,5.0,4.0,4.0,3.0,4.0,3.0,5.0,5.0,3.0
537,4.0,4.0,4.0,2.0,1.0,3.0,2.0,2.0,1.0,1.0,...,4.0,4.0,3.0,NaN,NaN,NaN,1.0,1.0,4.0,1.0
416,5.0,5.0,5.0,5.0,4.0,5.0,5.0,5.0,4.0,5.0,...,NaN,2.0,5.0,4.0,4.0,5.0,2.0,3.0,5.0,3.0
303,5.0,4.0,5.0,5.0,4.0,5.0,4.0,5.0,1.0,3.0,...,NaN,5.0,NaN,3.0,5.0,5.0,1.0,5.0,4.0,NaN
234,4.0,2.0,4.0,3.0,3.0,3.0,3.0,3.0,3.0,NaN,...,3.0,3.0,3.0,3.0,3.0,5.0,NaN,3.0,4.0,2.0


In [7]:
data_nmpy=data.to_numpy()
data_nmpy=data_nmpy.astype(np.float32)
data_nmpy

array([[ 5., nan, nan, ...,  3.,  5., nan],
       [ 4.,  2.,  3., ..., nan,  4., nan],
       [ 5.,  4.,  5., ...,  5.,  5.,  3.],
       ...,
       [ 5.,  4., nan, ...,  5.,  4., nan],
       [ 4.,  3.,  4., ..., nan,  5.,  3.],
       [ 5.,  3.,  5., ..., nan, nan, nan]], dtype=float32)

In [8]:
mask_real=np.logical_not(np.isnan(data_nmpy))
theta=6
dshape = data_nmpy.shape
users=tf.Variable(np.random.uniform(low=0.0, high=1.0, size=(dshape[0],theta)).astype(np.float32))
films=tf.Variable(np.random.uniform(low=0.0, high=1.0, size=(theta,dshape[1])).astype(np.float32))
opt1 = tf.keras.optimizers.Adam(learning_rate=0.001)
opt2 = tf.keras.optimizers.Adam(learning_rate=0.001)
loss=tf.keras.losses.MeanSquaredError()
def lossFunc(y_pred):
    y_target = tf.boolean_mask(data_nmpy, mask_real)
    y_pred=tf.boolean_mask(y_pred, mask_real)
    l=loss(y_target,y_pred)
    return l

In [9]:
for e in range(3000):
    with tf.GradientTape() as tapef ,tf.GradientTape() as tapeu:
        y_pred=tf.matmul(users,films)
        l=lossFunc(y_pred)
    if e%200==0:
        print('\n\nepoch : '+str(e),l)
        
    gf=tapef.gradient(l,films)
    gu=tapeu.gradient(l,users)
    opt1.apply_gradients([(gf, films)]) 
    opt2.apply_gradients([(gu, users)])



epoch : 0 tf.Tensor(6.5737605, shape=(), dtype=float32)
Instructions for updating:
Use tf.identity instead.


epoch : 200 tf.Tensor(2.4301202, shape=(), dtype=float32)


epoch : 400 tf.Tensor(1.1160065, shape=(), dtype=float32)


epoch : 600 tf.Tensor(0.84357876, shape=(), dtype=float32)


epoch : 800 tf.Tensor(0.78300935, shape=(), dtype=float32)


epoch : 1000 tf.Tensor(0.76490474, shape=(), dtype=float32)


epoch : 1200 tf.Tensor(0.75499755, shape=(), dtype=float32)


epoch : 1400 tf.Tensor(0.74638975, shape=(), dtype=float32)


epoch : 1600 tf.Tensor(0.7376487, shape=(), dtype=float32)


epoch : 1800 tf.Tensor(0.72821456, shape=(), dtype=float32)


epoch : 2000 tf.Tensor(0.7175803, shape=(), dtype=float32)


epoch : 2200 tf.Tensor(0.7052357, shape=(), dtype=float32)


epoch : 2400 tf.Tensor(0.69081324, shape=(), dtype=float32)


epoch : 2600 tf.Tensor(0.6743514, shape=(), dtype=float32)


epoch : 2800 tf.Tensor(0.6564826, shape=(), dtype=float32)


In [10]:
users_=users.numpy()
films_=films.numpy()
films_=films_.reshape((films_.shape[1],films_.shape[0]))

In [11]:
engine = create_engine('mysql+pymysql://root:@localhost:3306/my_data')
try:
    statement1 = text("""DROP TABLE users_latent""")
    engine.execute(statement1)

except:
    pass

stat="""CREATE TABLE users_latent ( id INT NOT NULL"""
for c in range( users_.shape[1] ):
    stat+=f", l{c} DOUBLE"
stat=stat+')'
stat
engine.execute(stat)

for x in range(users_.shape[0]):
    lat=users_[x].tolist()

    ff={"id":int(data.index[x])}
    for l in range(len(lat)):
        ff['l'+str(l)]=lat[l]

    st1='(id,'
    st2='(:id,'
    for l in range(len(lat)):
        st1+='l'+str(l)+','
        st2+=':l'+str(l)+','
        
    st1=st1[:-1]+')'
    st2=st2[:-1]+')'

    statement=text(f"INSERT INTO users_latent{st1} VALUES {st2}")
    engine.execute(statement,ff)


In [12]:
try:
    statement1 = text("""DROP TABLE films_latent""")
    engine.execute(statement1)

except:
    pass

stat="""CREATE TABLE films_latent ( film TEXT NOT NULL"""
for c in range( users_.shape[1] ):
    stat+=f", l{c} DOUBLE"
stat=stat+')'
stat
engine.execute(stat)



for x in range(films_.shape[0]):
    lat=films_[x].tolist()

    ff={"film":data.columns[x]}
    for l in range(len(lat)):
        ff['l'+str(l)]=lat[l]

    st1='(film,'
    st2='(:film,'
    for l in range(len(lat)):
        st1+='l'+str(l)+','
        st2+=':l'+str(l)+','
        
    st1=st1[:-1]+')'
    st2=st2[:-1]+')'

    statement=text(f"INSERT INTO films_latent{st1} VALUES {st2}")
    engine.execute(statement,ff)


In [13]:

with engine.begin() as con:
    
    try: #if exist delete it , if not pass the exception
        statement1 = text("""DROP TABLE pivot""")
        con.execute(statement1)
    except:
        pass
    
    data.to_sql("pivot", con = engine,if_exists="replace",index='user_id')